In [ ]:
##### QA query   KSSB_01_2nd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
import key

# 1. LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=key.key["OPEN_API_KEY"])

# 2. 질문 생성 프롬프트
question_prompt = ChatPromptTemplate.from_template("""
다음 문서를 기반으로 적절한 질문을 하나 생성하세요. 답은 반드시 문서 안에 있어야 합니다. 모든 질문은 한국어로 생성하세요.

문서:
{context}

질문:
""")

# 3. 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_template("""
질문에 대해 아래 문서를 참고해서 최대한 정확하게 답변하세요. 문서에 명시되지 않은 내용은 답하지 마세요. 모든 답변은 한국어로 하세요.

문서:
{context}

질문:
{question}

답변:
""")


qa_results = []

for doc in KSSB_01_2nd :  
    context = doc.page_content
    chunk_id = doc.metadata["chunk_id"]
    source = doc.metadata["source"]

    # 1. 질문 생성
    question = llm.invoke(question_prompt.format(context=context)).content.strip()

    # 2. 답변 생성
    answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

    # 3. 저장
    qa_results.append({
        "chunk_id": chunk_id,
        "source": source,
        "question": question,
        "answer": answer,
        "context": context[:300] + "..."  # 긴 context는 요약 출력
    })


print(len(qa_results))
print(qa_results)


##### 저장하기 json / vector
import json

# 파일로 저장
with open("KSSB_01_2nd_qa_results.json", "w", encoding="utf-8") as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

print("✅ KSSB_01_2nd_qa_results.json 파일 저장 완료")


# 저장하기 vector
from langchain.schema import Document

qa_documents = []

for item in qa_results:
    qa_documents.append(Document(
        page_content = f"Q: {item['question']}\nA: {item['answer']}",
        metadata = {
            "question": item["question"],
            "chunk_id": item["chunk_id"],
            "source": item["source"]
        }
    ))

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import key

# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# 벡터스토어로 저장
qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
qa_vectorstore.save_local("vectorstoresqa/KSSB_01_2nd_qa_results_openai")

print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/KSSB_01_2nd_qa_results_openai")



##### 평가하기    KSSB_01_2nd
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI
import key

# ✅ 평가에 사용할 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])


# ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
evaluator = load_evaluator("context_qa", llm=llm)

# ✅ 평가 실행
evaluated_results = []

for item in qa_results:
    result = evaluator.evaluate_strings(
        input={"question": item["question"], "context": item["context"]},
        prediction=item["answer"],
        reference=item["answer"]  # ⬅️ 추가됨
    )
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "context": item["context"],
        "score": result["score"],
        "reasoning": result.get("reasoning", "")
    })


# 결과 출력(상위 5개)
import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
display(df_eval[["chunk_id", "score", "question", "answer"]].head())


#점수 구체화
evaluator = load_evaluator(
    evaluator="score_string",
    criteria={"relevance": "질문과 답변의 관련성"},
    normalize_by=10,
    llm=llm
)

# DataFrame으로 변환
df_eval = pd.DataFrame(evaluated_results)

# CSV로 저장
df_eval.to_csv("KSSB_01_2nd_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

print("✅ LLM 평가 결과 저장 완료: KSSB_01_2nd_qa_llm_eval_results.csv")


# 코사인 유사도 검사
from langchain_openai import OpenAIEmbeddings
import key

embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


evaluated_results = []

for item in qa_results:
    try:
        answer_emb = embedding_model.embed_query(item["answer"])
        context_emb = embedding_model.embed_query(item["context"])
        score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
    except Exception as e:
        print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
        score = None
    
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "score_cosine": score
    })

import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


import pandas as pd

# 1. DataFrame으로 변환
df = pd.DataFrame(evaluated_results)

# 2. CSV로 저장
df.to_csv("KSSB_01_2nd_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

print("✅ CSV 파일 저장 완료: KSSB_01_2nd_qa_results_cosine_eval.csv")

78
[{'chunk_id': '지속가능성 공시기준서 제1호_목적_01', 'source': '지속가능성 공시기준서 제1호_목적', 'question': '지속가능성 관련 위험 및 기회에 대한 정보를 공시하는 목적은 무엇인가요?', 'answer': '지속가능성 관련 위험 및 기회에 대한 정보를 공시하는 목적은 주요이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한 기업의 지속가능성 관련 위험 및 기회에 대한 정보를 제공하기 위함입니다.', 'context': '목적 1 지속가능성 공시기준서 제1호 지속가능성 관련 재무정보 공시를 위 ‘ 한 일반사항의 ’ 목적은 일반목적재무보고서의 주요이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한기업의 지속가 , 능성 관련위험 및 기회에 대한 정보를 기업이 공시하도록 하는 것이다.1) 2 지속가능성 관련 위험 및 기회에 대한 정보는 주요이용자에게 유 용하다 이는 단기 중기 및 장기에 걸쳐 현금흐름을 창출하는 기 . , 업의 능력이 기업 가치사슬전반에 걸쳐 기업과 기업의 이해관계자 , 사회경제 및 자연환경 간의 상호작용과 불가분하게 연계되...'}, {'chunk_id': '지속가능성 공시기준서 제1호_적용범위_01', 'source': '지속가능성 공시기준서 제1호_적용범위', 'question': '기업이 지속가능성 관련 재무공시를 작성할 때 어떤 정보를 공시해야 하는가요?', 'answer': '기업이 지속가능성 관련 재무공시를 작성할 때, 기업은 기업 전망에 영향을 미칠 것으로 합리적으로 예상할 수 있는 기후 관련 위험 및 기회에 대한 정보를 공시해야 합니다. 또한, 기업이 기업 전망에 영향을 미칠 것으로 합리적으로 예상할 수 있는 기후 외 다른 지속가능성 관련 위험 및 기회에 대한 정보도 선택하여 공시할 수 있습니다. (예: 안전보건, 생물다양성, 인적자본 등)', 'context': '적용범위 5 이 기준서는 한국 지속가능성 공시기준이하 지속가능성 공시기 ( ‘ 준’)에 따라 지속가능

,chunk_id,score,question,answer
0,지속가능성 공시기준서 제1호_목적_01,1,지속가능성 관련 위험 및 기회에 대한 정보를 공시하는 목적은 무엇인가요?,지속가능성 관련 위험 및 기회에 대한 정보를 공시하는 목적은 주요이용자가 기업에 대...
1,지속가능성 공시기준서 제1호_적용범위_01,1,기업이 지속가능성 관련 재무공시를 작성할 때 어떤 정보를 공시해야 하는가요?,"기업이 지속가능성 관련 재무공시를 작성할 때, 기업은 기업 전망에 영향을 미칠 것으..."
2,지속가능성 공시기준서 제1호_개념적 기반_01,1,지속가능성 관련 재무정보가 유용하게 되기 위해서는 어떤 조건을 충족해야 하는가요?,지속가능성 관련 재무정보가 유용하게 되기 위해서는 비교가능하고 검증가능하며 적시성 ...
3,지속가능성 공시기준서 제1호_공정한 표시_01,1,기업이 지속가능성 관련 위험 및 기회를 공정하게 표시하기 위해 어떤 원칙을 따라야 ...,기업이 지속가능성 관련 위험 및 기회를 공정하게 표시하기 위해서는 공정한 표시를 위...
4,지속가능성 공시기준서 제1호_중요성_01,1,기업이 지속가능성 관련 위험과 기회에 대한 정보를 공시하는 이유는 무엇인가요?,기업이 지속가능성 관련 위험과 기회에 대한 정보를 공시하는 이유는 기업 전망에 영향...


This chain was only tested with GPT-4. Performance may be significantly worse with other models.


✅ LLM 평가 결과 저장 완료: KSSB_01_2nd_qa_llm_eval_results.csv


,chunk_id,score_cosine,question
59,지속가능성 공시기준서 제1호_부속지침_주요이용자_01,0.992046,주요이용자 IG1 지속가능성 공시기준서 제호의 목적은 무엇인가요?
5,지속가능성 공시기준서 제1호_보고기업_01,0.987393,보고기업 20 지속가능성 관련 재무공시의 보고기업은 무엇과 동일해야 한가요? (문단...
28,지속가능성 공시기준서 제1호_부록 B. 적용지침_06,0.972836,B20 기업이 지속가능성 관련 위험 또는 기회를 식별하기 위해 어떤 요구사항을 적용...
69,지속가능성 공시기준서 제1호_부속지침_프레임워크 적용지침 CDSB IG25_01,0.961564,기업이 지속가능성 관련 위험 및 기회를 식별할 때 어떤 가이드라인을 참고할 수 있는...
13,지속가능성 공시기준서 제1호_보고시기_02,0.961497,어떤 경우에 기업이 중간 일반목적재무보고서를 발표하도록 요구받을 수 있나요?
61,지속가능성 공시기준서 제1호_부속지침_주요이용자의 정보수요 충족_02,0.960508,IG6에서 언급된 정보수요의 평가 방법은 무엇인가요?
1,지속가능성 공시기준서 제1호_적용범위_01,0.958754,기업이 지속가능성 관련 재무공시를 작성할 때 어떤 정보를 공시해야 하는가요?
34,지속가능성 공시기준서 제1호_부록 B. 적용지침_12,0.958425,기업이 연계된 정보를 제공할 때 어떤 내용을 설명해야 할 수 있을까요?
9,지속가능성 공시기준서 제1호_지침의 원천_01,0.956078,기업이 지속가능성 관련 위험과 기회를 식별할 때 참고할 수 있는 기준은 무엇인가요?
32,지속가능성 공시기준서 제1호_부록 B. 적용지침_10,0.951947,기업이 지속가능성 관련 재무공시를 할 때 면제규정을 사용할 수 있는 경우는 어떤 경...


✅ CSV 파일 저장 완료: KSSB_01_2nd_qa_results_cosine_eval.csv


In [ ]:
##### QA query   KSSB_02_2nd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
import key


# 1. LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=key.key["OPEN_API_KEY"])

# 2. 질문 생성 프롬프트
question_prompt = ChatPromptTemplate.from_template("""
다음 문서를 기반으로 적절한 질문을 하나 생성하세요. 답은 반드시 문서 안에 있어야 합니다. 모든 질문은 한국어로 생성하세요.

문서:
{context}

질문:
""")

# 3. 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_template("""
질문에 대해 아래 문서를 참고해서 최대한 정확하게 답변하세요. 문서에 명시되지 않은 내용은 답하지 마세요. 모든 답변은 한국어로 하세요.

문서:
{context}

질문:
{question}

답변:
""")


qa_results = []

for doc in KSSB_02_2nd :  
    context = doc.page_content
    chunk_id = doc.metadata["chunk_id"]
    source = doc.metadata["source"]

    # 1. 질문 생성
    question = llm.invoke(question_prompt.format(context=context)).content.strip()

    # 2. 답변 생성
    answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

    # 3. 저장
    qa_results.append({
        "chunk_id": chunk_id,
        "source": source,
        "question": question,
        "answer": answer,
        "context": context[:300] + "..."  # 긴 context는 요약 출력
    })


print(len(qa_results))
print(qa_results)


##### 저장하기 json / vector
import json

# 파일로 저장
with open("KSSB_02_2nd_qa_results.json", "w", encoding="utf-8") as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

print("✅ KSSB_02_2nd_qa_results.json 파일 저장 완료")


# 저장하기 vector
from langchain.schema import Document

qa_documents = []

for item in qa_results:
    qa_documents.append(Document(
        page_content = f"Q: {item['question']}\nA: {item['answer']}",
        metadata = {
            "question": item["question"],
            "chunk_id": item["chunk_id"],
            "source": item["source"]
        }
    ))

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import key

# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# 벡터스토어로 저장
qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
qa_vectorstore.save_local("vectorstoresqa/KSSB_02_2nd_qa_results_openai")

print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/KSSB_02_2nd_qa_results_openai")



##### 평가하기    KSSB_02_2nd
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI
import key

# ✅ 평가에 사용할 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])


# ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
evaluator = load_evaluator("context_qa", llm=llm)

# ✅ 평가 실행
evaluated_results = []

for item in qa_results:
    result = evaluator.evaluate_strings(
        input={"question": item["question"], "context": item["context"]},
        prediction=item["answer"],
        reference=item["answer"]  # ⬅️ 추가됨
    )
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "context": item["context"],
        "score": result["score"],
        "reasoning": result.get("reasoning", "")
    })


# 결과 출력(상위 5개)
import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
display(df_eval[["chunk_id", "score", "question", "answer"]].head())


#점수 구체화
evaluator = load_evaluator(
    evaluator="score_string",
    criteria={"relevance": "질문과 답변의 관련성"},
    normalize_by=10,
    llm=llm
)

# DataFrame으로 변환
df_eval = pd.DataFrame(evaluated_results)

# CSV로 저장
df_eval.to_csv("KSSB_02_2nd_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

print("✅ LLM 평가 결과 저장 완료: KSSB_02_2nd_qa_llm_eval_results.csv")


# 코사인 유사도 검사
from langchain_openai import OpenAIEmbeddings
import key

embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


evaluated_results = []

for item in qa_results:
    try:
        answer_emb = embedding_model.embed_query(item["answer"])
        context_emb = embedding_model.embed_query(item["context"])
        score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
    except Exception as e:
        print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
        score = None
    
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "score_cosine": score
    })

import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


import pandas as pd

# 1. DataFrame으로 변환
df = pd.DataFrame(evaluated_results)

# 2. CSV로 저장
df.to_csv("KSSB_02_2nd_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

print("✅ CSV 파일 저장 완료: KSSB_02_2nd_qa_results_cosine_eval.csv")

129
[{'chunk_id': '지속가능성 공시기준서 제2호_목적_01', 'source': '지속가능성 공시기준서 제2호_목적', 'question': '기후 관련 공시사항의 목적은 무엇인가요?', 'answer': '기후 관련 공시사항의 목적은 기업의 주요 이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한 기업의 기후 관련 위험 및 기회에 대한 정보를 기업이 공시하도록 요구하는 것입니다.', 'context': '목적\n1\n지속가능성 공시기준서 제호\n2\n기후 관련 공시사항의 목적은 일반\n ’\n‘\n목적재무보고서의 주요이용자가 기업에 대한 자원 제공과 관련된 \n의사결정을 할 때 유용한\n기업의 \n, \n기후 관련 위험 및 기회에 대한 \n정보를 기업이 공시하도록 요구하는 것이다.1)  \n2\n이 기준서는 단기\n중기 또는 장기에 걸쳐 기업의 현금흐름\n자금\n, \n, \n조달 접근성 또는 자본비용에 영향을 미칠 것으로 합리적으로 예\n상할 수 있는 기후 관련 위험 및 기회에 대한 정보를 공시하도록 \n요구한다이 기준서의 목적상\n이러한 위험 및 기회를 기업 전망\n....'}, {'chunk_id': '지속가능성 공시기준서 제2호_적용범위_01', 'source': '지속가능성 공시기준서 제2호_적용범위', 'question': '기업이 노출된 기후 관련 위험에는 어떤 종류의 위험이 포함되어 있나요?', 'answer': '기업이 노출된 기후 관련 위험에는 기후 관련 물리적 위험과 기후 관련 전환 위험이 포함됩니다.', 'context': '적용범위\n3\n이 기준서는 다음에 적용한다. \n⑴기업이 노출된 다음과 같은 기후 관련 위험\n㈎기후 관련 물리적 위험\n㈏기후 관련 전환 위험\n⑵기업이 이용할 수 있는 기후 관련 기회\n3.1\n기업이 기업 전망에 영향을 미칠 것으로 합리적으로 예상할 수 \n있는 기후 관련 위험 및 기회에 대한 모든 정보가 중요하지 않다\n고 판단하여 이 기준서에 따른 공시를 생략한 경우\n기업은 

,chunk_id,score,question,answer
0,지속가능성 공시기준서 제2호_목적_01,1,기후 관련 공시사항의 목적은 무엇인가요?,기후 관련 공시사항의 목적은 기업의 주요 이용자가 기업에 대한 자원 제공과 관련된 ...
1,지속가능성 공시기준서 제2호_적용범위_01,1,기업이 노출된 기후 관련 위험에는 어떤 종류의 위험이 포함되어 있나요?,기업이 노출된 기후 관련 위험에는 기후 관련 물리적 위험과 기후 관련 전환 위험이 ...
2,지속가능성 공시기준서 제2호_핵심요소_01,1,핵심요소란 무엇이며 왜 중요한가요?,핵심요소란 어떤 시스템이나 문제 해결에 있어서 가장 중요하고 핵심적인 부분을 말합니...
3,지속가능성 공시기준서 제2호_거버넌스_01,1,기업이 기후 관련 위험 및 기회를 모니터링하고 관리하기 위해 거버넌스 프로세스를 사...,기업이 기후 관련 위험 및 기회를 모니터링하고 관리하기 위해 거버넌스 프로세스를 사...
4,지속가능성 공시기준서 제2호_전략_01,1,기후 관련 위험과 기회에 대한 정보를 어떻게 안내받는지와 얼마나 자주 안내받는지에 ...,의사결정기구들이 기후 관련 위험과 기회에 대한 정보를 어떻게 안내받을지와 얼마나 자...


This chain was only tested with GPT-4. Performance may be significantly worse with other models.


✅ LLM 평가 결과 저장 완료: KSSB_02_2nd_qa_llm_eval_results.csv


,chunk_id,score_cosine,question
3,지속가능성 공시기준서 제2호_거버넌스_01,0.947971,기업이 기후 관련 위험 및 기회를 모니터링하고 관리하기 위해 거버넌스 프로세스를 사...
114,지속가능성 공시기준서 제2호_부속지침_온실가스 배출량의 통합 및 세분화_01,0.947191,온실가스 배출량 공시를 수행할 때 정보 세분화의 필요성을 판단하는 과정에서 고려해야...
113,지속가능성 공시기준서 제2호_부속지침_예시사례_01,0.941560,"예시사례는 무엇을 보여주는 것이며, 어떤 목적을 가지고 있는가?"
91,지속가능성 공시기준서 제2호_부록 B 적용지침_10,0.941188,기업이 보고기간과 상이한 보고기간의 정보를 이용하여 온실가스 배출량을 측정할 수 있...
40,지속가능성 공시기준서 제2호_지표 및 목표_25,0.940212,기업이 자신의 온실가스 배출량을 측정하는 경우 어떤 기준을 사용해야 하나요?
36,지속가능성 공시기준서 제2호_지표 및 목표_21,0.938408,기업의 회복력을 평가하는 데 어떤 정보가 필요한가요?
0,지속가능성 공시기준서 제2호_목적_01,0.937721,기후 관련 공시사항의 목적은 무엇인가요?
92,지속가능성 공시기준서 제2호_부록 B 적용지침_11,0.936665,기업이 자신의 온실가스 배출량을 측정할 때 어떤 기준을 사용해야 하나요?
57,지속가능성 공시기준서 제2호_지표 및 목표_42,0.936242,기업이 온실가스 순배출량 목표를 공시할 때 주의해야 하는 사항은 무엇인가요?
35,지속가능성 공시기준서 제2호_지표 및 목표_20,0.935961,기업이 기후 관련 시나리오 분석에 사용할 투입변수를 선택할 때 고려해야 하는 요소는...


✅ CSV 파일 저장 완료: KSSB_02_2nd_qa_results_cosine_eval.csv


In [ ]:
##### QA query   KSSB_101_2nd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
import key


# 1. LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=key.key["OPEN_API_KEY"])

# 2. 질문 생성 프롬프트
question_prompt = ChatPromptTemplate.from_template("""
다음 문서를 기반으로 적절한 질문을 하나 생성하세요. 답은 반드시 문서 안에 있어야 합니다. 모든 질문은 한국어로 생성하세요.

문서:
{context}

질문:
""")

# 3. 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_template("""
질문에 대해 아래 문서를 참고해서 최대한 정확하게 답변하세요. 문서에 명시되지 않은 내용은 답하지 마세요. 모든 답변은 한국어로 하세요.

문서:
{context}

질문:
{question}

답변:
""")


qa_results = []

for doc in KSSB_101_2nd :  
    context = doc.page_content
    chunk_id = doc.metadata["chunk_id"]
    source = doc.metadata["source"]

    # 1. 질문 생성
    question = llm.invoke(question_prompt.format(context=context)).content.strip()

    # 2. 답변 생성
    answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

    # 3. 저장
    qa_results.append({
        "chunk_id": chunk_id,
        "source": source,
        "question": question,
        "answer": answer,
        "context": context[:300] + "..."  # 긴 context는 요약 출력
    })


print(len(qa_results))
print(qa_results)


##### 저장하기 json / vector
import json

# 파일로 저장
with open("KSSB_101_2nd_qa_results.json", "w", encoding="utf-8") as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

print("✅ KSSB_101_2nd_qa_results.json 파일 저장 완료")


# 저장하기 vector
from langchain.schema import Document

qa_documents = []

for item in qa_results:
    qa_documents.append(Document(
        page_content = f"Q: {item['question']}\nA: {item['answer']}",
        metadata = {
            "question": item["question"],
            "chunk_id": item["chunk_id"],
            "source": item["source"]
        }
    ))

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import key

# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# 벡터스토어로 저장
qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
qa_vectorstore.save_local("vectorstoresqa/KSSB_101_2nd_qa_results_openai")

print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/KSSB_101_2nd_qa_results_openai")



##### 평가하기    KSSB_101_2nd
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI
import key

# ✅ 평가에 사용할 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])


# ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
evaluator = load_evaluator("context_qa", llm=llm)

# ✅ 평가 실행
evaluated_results = []

for item in qa_results:
    result = evaluator.evaluate_strings(
        input={"question": item["question"], "context": item["context"]},
        prediction=item["answer"],
        reference=item["answer"]  # ⬅️ 추가됨
    )
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "context": item["context"],
        "score": result["score"],
        "reasoning": result.get("reasoning", "")
    })


# 결과 출력(상위 5개)
import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
display(df_eval[["chunk_id", "score", "question", "answer"]].head())


#점수 구체화
evaluator = load_evaluator(
    evaluator="score_string",
    criteria={"relevance": "질문과 답변의 관련성"},
    normalize_by=10,
    llm=llm
)

# DataFrame으로 변환
df_eval = pd.DataFrame(evaluated_results)

# CSV로 저장
df_eval.to_csv("KSSB_101_2nd_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

print("✅ LLM 평가 결과 저장 완료: KSSB_101_2nd_qa_llm_eval_results.csv")


# 코사인 유사도 검사
from langchain_openai import OpenAIEmbeddings
import key

embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


evaluated_results = []

for item in qa_results:
    try:
        answer_emb = embedding_model.embed_query(item["answer"])
        context_emb = embedding_model.embed_query(item["context"])
        score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
    except Exception as e:
        print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
        score = None
    
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "score_cosine": score
    })

import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


import pandas as pd

# 1. DataFrame으로 변환
df = pd.DataFrame(evaluated_results)

# 2. CSV로 저장
df.to_csv("KSSB_101_2nd_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

print("✅ CSV 파일 저장 완료: KSSB_101_2nd_qa_results_cosine_eval.csv")

129
[{'chunk_id': '지속가능성 공시기준서 제2호_목적_01', 'source': '지속가능성 공시기준서 제2호_목적', 'question': '기후 관련 공시사항의 목적은 무엇인가요?', 'answer': '기후 관련 공시사항의 목적은 기업이 기후 관련 위험 및 기회에 대한 정보를 공시하여, 주요이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한 정보를 제공하는 것입니다.', 'context': '목적\n1\n지속가능성 공시기준서 제호\n2\n기후 관련 공시사항의 목적은 일반\n ’\n‘\n목적재무보고서의 주요이용자가 기업에 대한 자원 제공과 관련된 \n의사결정을 할 때 유용한\n기업의 \n, \n기후 관련 위험 및 기회에 대한 \n정보를 기업이 공시하도록 요구하는 것이다.1)  \n2\n이 기준서는 단기\n중기 또는 장기에 걸쳐 기업의 현금흐름\n자금\n, \n, \n조달 접근성 또는 자본비용에 영향을 미칠 것으로 합리적으로 예\n상할 수 있는 기후 관련 위험 및 기회에 대한 정보를 공시하도록 \n요구한다이 기준서의 목적상\n이러한 위험 및 기회를 기업 전망\n....'}, {'chunk_id': '지속가능성 공시기준서 제2호_적용범위_01', 'source': '지속가능성 공시기준서 제2호_적용범위', 'question': '기업이 노출된 기후 관련 위험에는 어떤 종류가 포함되어 있나요?', 'answer': '기업이 노출된 기후 관련 위험에는 기후 관련 물리적 위험과 기후 관련 전환 위험이 포함됩니다.', 'context': '적용범위\n3\n이 기준서는 다음에 적용한다. \n⑴기업이 노출된 다음과 같은 기후 관련 위험\n㈎기후 관련 물리적 위험\n㈏기후 관련 전환 위험\n⑵기업이 이용할 수 있는 기후 관련 기회\n3.1\n기업이 기업 전망에 영향을 미칠 것으로 합리적으로 예상할 수 \n있는 기후 관련 위험 및 기회에 대한 모든 정보가 중요하지 않다\n고 판단하여 이 기준서에 따른 공시를 생략한 경우\n기업은 그러\n, \n한

,chunk_id,score,question,answer
0,지속가능성 공시기준서 제2호_목적_01,1,기후 관련 공시사항의 목적은 무엇인가요?,기후 관련 공시사항의 목적은 기업이 기후 관련 위험 및 기회에 대한 정보를 공시하여...
1,지속가능성 공시기준서 제2호_적용범위_01,1,기업이 노출된 기후 관련 위험에는 어떤 종류가 포함되어 있나요?,기업이 노출된 기후 관련 위험에는 기후 관련 물리적 위험과 기후 관련 전환 위험이 ...
2,지속가능성 공시기준서 제2호_핵심요소_01,1,핵심요소란 무엇을 의미하나요?,핵심요소란 어떤 시스템이나 개념의 중심이 되는 중요한 구성 요소를 말합니다.
3,지속가능성 공시기준서 제2호_거버넌스_01,1,기업이 기후 관련 위험 및 기회를 모니터링하고 관리하기 위해 거버넌스 프로세스를 사...,기업이 기후 관련 위험 및 기회를 모니터링하고 관리하기 위해 거버넌스 프로세스를 사...
4,지속가능성 공시기준서 제2호_전략_01,1,기후 관련 위험과 기회에 대한 정보를 어떻게 안내받는지에 대한 프로세스는 무엇인가요?,기후 관련 위험과 기회에 대한 정보를 안내받는 프로세스는 ㈐의사결정기구들이나 개인들...


This chain was only tested with GPT-4. Performance may be significantly worse with other models.


✅ LLM 평가 결과 저장 완료: KSSB_101_2nd_qa_llm_eval_results.csv


,chunk_id,score_cosine,question
123,지속가능성 공시기준서 제2호_부속지침_자산운용 부문의 금융배출량 세분화_01,0.953141,자산운용 부문의 금융배출량 세분화를 위해 어떤 기준이 적용되었는지 설명해 주세요.
9,지속가능성 공시기준서 제2호_위험관리_05,0.951993,기업의 사업모형에 대한 현재 변화 및 예상 변화를 다루는 공개초안에서 어떤 내용들이...
113,지속가능성 공시기준서 제2호_부속지침_예시사례_01,0.951534,지속가능성 공시기준서 제호의 측면을 보여주는 예시사례는 무엇인가요?
4,지속가능성 공시기준서 제2호_전략_01,0.950229,기후 관련 위험과 기회에 대한 정보를 어떻게 안내받는지에 대한 프로세스는 무엇인가요?
3,지속가능성 공시기준서 제2호_거버넌스_01,0.948157,기업이 기후 관련 위험 및 기회를 모니터링하고 관리하기 위해 거버넌스 프로세스를 사...
40,지속가능성 공시기준서 제2호_지표 및 목표_25,0.938688,기업이 온실가스 배출량을 측정하는 경우 어떤 기준을 사용해야 하나요?
92,지속가능성 공시기준서 제2호_부록 B 적용지침_11,0.938539,"기업이 자신의 온실가스 배출량을 측정하는 경우, 어떤 기준을 사용하여 일곱가지 온실..."
88,지속가능성 공시기준서 제2호_부록 B 적용지침_07,0.938066,기업의 회복력을 평가하는 데 어떤 정보가 필요한가요?
114,지속가능성 공시기준서 제2호_부속지침_온실가스 배출량의 통합 및 세분화_01,0.933531,온실가스 배출량 공시를 위해 정보를 세분화해야 하는 이유에 대해 어떤 사례가 제시되...
0,지속가능성 공시기준서 제2호_목적_01,0.931836,기후 관련 공시사항의 목적은 무엇인가요?


✅ CSV 파일 저장 완료: KSSB_101_2nd_qa_results_cosine_eval.csv


In [ ]:
##### QA query   KOR_IFRS_S1_2nd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
import key


# 1. LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=key.key["OPEN_API_KEY"])

# 2. 질문 생성 프롬프트
question_prompt = ChatPromptTemplate.from_template("""
다음 문서를 기반으로 적절한 질문을 하나 생성하세요. 답은 반드시 문서 안에 있어야 합니다. 모든 질문은 한국어로 생성하세요.

문서:
{context}

질문:
""")

# 3. 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_template("""
질문에 대해 아래 문서를 참고해서 최대한 정확하게 답변하세요. 문서에 명시되지 않은 내용은 답하지 마세요. 모든 답변은 한국어로 하세요.

문서:
{context}

질문:
{question}

답변:
""")


qa_results = []

for doc in KOR_IFRS_S1_2nd :  
    context = doc.page_content
    chunk_id = doc.metadata["chunk_id"]
    source = doc.metadata["source"]

    # 1. 질문 생성
    question = llm.invoke(question_prompt.format(context=context)).content.strip()

    # 2. 답변 생성
    answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

    # 3. 저장
    qa_results.append({
        "chunk_id": chunk_id,
        "source": source,
        "question": question,
        "answer": answer,
        "context": context[:300] + "..."  # 긴 context는 요약 출력
    })


print(len(qa_results))
print(qa_results)


##### 저장하기 json / vector
import json

# 파일로 저장
with open("KOR_IFRS_S1_2nd_qa_results.json", "w", encoding="utf-8") as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

print("✅ KOR_IFRS_S1_2nd_qa_results.json 파일 저장 완료")


# 저장하기 vector
from langchain.schema import Document

qa_documents = []

for item in qa_results:
    qa_documents.append(Document(
        page_content = f"Q: {item['question']}\nA: {item['answer']}",
        metadata = {
            "question": item["question"],
            "chunk_id": item["chunk_id"],
            "source": item["source"]
        }
    ))

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import key

# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# 벡터스토어로 저장
qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
qa_vectorstore.save_local("vectorstoresqa/KOR_IFRS_S1_2nd_qa_results_openai")

print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/KOR_IFRS_S1_2nd_qa_results_openai")



##### 평가하기    KOR_IFRS_S1_2nd
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI
import key

# ✅ 평가에 사용할 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])


# ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
evaluator = load_evaluator("context_qa", llm=llm)

# ✅ 평가 실행
evaluated_results = []

for item in qa_results:
    result = evaluator.evaluate_strings(
        input={"question": item["question"], "context": item["context"]},
        prediction=item["answer"],
        reference=item["answer"]  # ⬅️ 추가됨
    )
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "context": item["context"],
        "score": result["score"],
        "reasoning": result.get("reasoning", "")
    })


# 결과 출력(상위 5개)
import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
display(df_eval[["chunk_id", "score", "question", "answer"]].head())


#점수 구체화
evaluator = load_evaluator(
    evaluator="score_string",
    criteria={"relevance": "질문과 답변의 관련성"},
    normalize_by=10,
    llm=llm
)

# DataFrame으로 변환
df_eval = pd.DataFrame(evaluated_results)

# CSV로 저장
df_eval.to_csv("KOR_IFRS_S1_2nd_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

print("✅ LLM 평가 결과 저장 완료: KOR_IFRS_S1_2nd_qa_llm_eval_results.csv")


# 코사인 유사도 검사
from langchain_openai import OpenAIEmbeddings
import key

embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


evaluated_results = []

for item in qa_results:
    try:
        answer_emb = embedding_model.embed_query(item["answer"])
        context_emb = embedding_model.embed_query(item["context"])
        score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
    except Exception as e:
        print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
        score = None
    
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "score_cosine": score
    })

import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


import pandas as pd

# 1. DataFrame으로 변환
df = pd.DataFrame(evaluated_results)

# 2. CSV로 저장
df.to_csv("KOR_IFRS_S1_2nd_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

print("✅ CSV 파일 저장 완료: KOR_IFRS_S1_2nd_qa_results_cosine_eval.csv")

169
[{'chunk_id': 'IFRS 지속가능성 공시기준 S1_목적_01', 'source': 'IFRS 지속가능성 공시기준 S1_목적', 'question': 'IFRS S1의 목적은 무엇인가요?', 'answer': 'IFRS S1의 목적은 일반목적재무보고서의 주요이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한, 기업의 지속가능성 관련 위험 및 기회에 대한 정보를 기업이 공시하도록 요구하는 것입니다.', 'context': '목적 1 IFRS S1 ‘지속가능성 관련 재무정보 공시를 위한 일반 요구사항’의 목적은 일반목적재무보고서의 주요이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한, 기업의 지속가능성 관련 위험 및 기회에 대한 정보를 기업이 공시하도록 요구하는 것이 다.1) 2 지속가능성 관련 위험 및 기회에 대한 정보는 주요이용자에게 유 용하다. 이는 단기, 중기 및 장기에 걸쳐 현금흐름을 창출하는 기 업의 능력이 기업 가치사슬전반에 걸쳐 기업과 기업의 이해관계자, 사회, 경제 및 자연환경 간의 상호작용과 불가분하게 연계되어 있기 때...'}, {'chunk_id': 'IFRS 지속가능성 공시기준 S1_적용범위_01', 'source': 'IFRS 지속가능성 공시기준 S1_적용범위', 'question': '기업이 특정 지속가능성 관련 위험 및 기회에 대해 공시해야 하는 정보를 명시하는 것은 어떤 지침에서 이루어지나요?', 'answer': '다른 IFRS 지속가능성 공시기준에서 기업이 특정 지속가능성 관련 위험 및 기회에 대해 공시해야 하는 정보를 명시합니다.', 'context': '적용범위 5 이 기준서는 IFRS 지속가능성 공시기준에 따라 지속가능성 관련 재무공시를 작성하고 보고할 때 적용한다. 6 기업 전망에 영향을 미칠 것으로 합리적으로 예상할 수 없는 지속 가능성 관련 위험 및 기회는 이 기준서의 적용범위에서 제외된다. 7 다른 IFRS 지속가능성 공시기준에서는 기업이 특정 지속가능성 관련 위험 및 기

,chunk_id,score,question,answer
0,IFRS 지속가능성 공시기준 S1_목적_01,1,IFRS S1의 목적은 무엇인가요?,IFRS S1의 목적은 일반목적재무보고서의 주요이용자가 기업에 대한 자원 제공과 관...
1,IFRS 지속가능성 공시기준 S1_적용범위_01,1,기업이 특정 지속가능성 관련 위험 및 기회에 대해 공시해야 하는 정보를 명시하는 것...,다른 IFRS 지속가능성 공시기준에서 기업이 특정 지속가능성 관련 위험 및 기회에 ...
2,IFRS 지속가능성 공시기준 S1_개념적 기반_01,1,지속가능성 관련 재무정보가 유용해지기 위해서는 어떤 조건을 충족해야 하는가요?,"지속가능성 관련 재무정보가 유용해지기 위해서는 비교가능하고, 검증가능하며, 적시성 ..."
3,IFRS 지속가능성 공시기준 S1_공정한 표시_01,1,기업이 공정한 표시를 위해 요구되는 정보의 특징은 무엇인가요?,"기업이 공정한 표시를 위해 요구되는 정보의 특징은 비교가능하고, 검증가능하며, 적시..."
4,IFRS 지속가능성 공시기준 S1_중요성_01,1,기업이 중요한 정보를 식별하고 공시하기 위해 어떤 가이드라인을 적용해야 하는가?,기업은 중요한 정보를 식별하고 공시하기 위해 문단 B13~B37을 적용해야 합니다.


This chain was only tested with GPT-4. Performance may be significantly worse with other models.


✅ LLM 평가 결과 저장 완료: KOR_IFRS_S1_2nd_qa_llm_eval_results.csv


,chunk_id,score_cosine,question
71,IFRS 지속가능성 공시기준 S1_부속지침_SASB 기준 IE2 사례 1과 2는 기...,0.982861,SASB 기준 IE2 사례 1과 2에서 어떤 요구사항들이 언급되었나요?
57,IFRS 지속가능성 공시기준 S1_부속지침_주요이용자_01,0.975898,IG1 IFRS S1의 목적은 무엇인가요?
138,IFRS 지속가능성 공시기준 S1_결론도출근거_지표 및 목표_02,0.975130,ISSB가 기업에게 요구하는 지속가능성 공시기준은 무엇인가요?
92,"IFRS 지속가능성 공시기준 S1_결론도출근거_글로벌 기준선, 그리고 관할권 및 규...",0.975075,글로벌 기준선을 수립하는 BC27 IFRS 지속가능성 공시기준의 목표는 무엇인가요?
147,IFRS 지속가능성 공시기준 S1_결론도출근거_지침의 원천_08,0.973261,기업이 지속가능성 관련 위험 또는 기회를 식별할 때 어떤 기준을 이용해야 하는가요?
148,IFRS 지속가능성 공시기준 S1_결론도출근거_지침의 원천_09,0.971991,기업이 GRI 기준이나 ESRS를 이용하여 작성한 보고서를 단순히 IFRS 지속가능...
143,IFRS 지속가능성 공시기준 S1_결론도출근거_지침의 원천_04,0.968784,SASB 기준과 IFRS S1의 유사성에 대해 어떤 내용이 문서에 나와 있나요?
156,IFRS 지속가능성 공시기준 S1_결론도출근거_비교정보_03,0.968423,ISSB가 '실무적으로 적용할 수 없는'의 정의를 명확히 하기 위해 어떤 결정을 내...
77,IFRS 지속가능성 공시기준 S1_결론도출근거_도입_01,0.966948,IFRS S1은 어떤 목적을 가지고 개발되었는가?
70,IFRS 지속가능성 공시기준 S1_부속지침_예시사례_01,0.962849,IFRS S1의 예시사례는 어떤 목적을 가지고 있는가요?


✅ CSV 파일 저장 완료: KOR_IFRS_S1_2nd_qa_results_cosine_eval.csv


In [ ]:
##### QA query   KOR_IFRS_S2_2nd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
import key


# 1. LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=key.key["OPEN_API_KEY"])

# 2. 질문 생성 프롬프트
question_prompt = ChatPromptTemplate.from_template("""
다음 문서를 기반으로 적절한 질문을 하나 생성하세요. 답은 반드시 문서 안에 있어야 합니다. 모든 질문은 한국어로 생성하세요.

문서:
{context}

질문:
""")

# 3. 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_template("""
질문에 대해 아래 문서를 참고해서 최대한 정확하게 답변하세요. 문서에 명시되지 않은 내용은 답하지 마세요. 모든 답변은 한국어로 하세요.

문서:
{context}

질문:
{question}

답변:
""")


qa_results = []

for doc in KOR_IFRS_S2_2nd :  
    context = doc.page_content
    chunk_id = doc.metadata["chunk_id"]
    source = doc.metadata["source"]

    # 1. 질문 생성
    question = llm.invoke(question_prompt.format(context=context)).content.strip()

    # 2. 답변 생성
    answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

    # 3. 저장
    qa_results.append({
        "chunk_id": chunk_id,
        "source": source,
        "question": question,
        "answer": answer,
        "context": context[:300] + "..."  # 긴 context는 요약 출력
    })


print(len(qa_results))
print(qa_results)


##### 저장하기 json / vector
import json

# 파일로 저장
with open("KOR_IFRS_S2_2nd_qa_results.json", "w", encoding="utf-8") as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

print("✅ KOR_IFRS_S2_2nd_qa_results.json 파일 저장 완료")


# 저장하기 vector
from langchain.schema import Document

qa_documents = []

for item in qa_results:
    qa_documents.append(Document(
        page_content = f"Q: {item['question']}\nA: {item['answer']}",
        metadata = {
            "question": item["question"],
            "chunk_id": item["chunk_id"],
            "source": item["source"]
        }
    ))

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import key

# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# 벡터스토어로 저장
qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
qa_vectorstore.save_local("vectorstoresqa/KOR_IFRS_S2_2nd_qa_results_openai")

print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/KOR_IFRS_S2_2nd_qa_results_openai")



##### 평가하기    KOR_IFRS_S2_2nd
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI
import key

# ✅ 평가에 사용할 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])


# ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
evaluator = load_evaluator("context_qa", llm=llm)

# ✅ 평가 실행
evaluated_results = []

for item in qa_results:
    result = evaluator.evaluate_strings(
        input={"question": item["question"], "context": item["context"]},
        prediction=item["answer"],
        reference=item["answer"]  # ⬅️ 추가됨
    )
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "context": item["context"],
        "score": result["score"],
        "reasoning": result.get("reasoning", "")
    })


# 결과 출력(상위 5개)
import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
display(df_eval[["chunk_id", "score", "question", "answer"]].head())


#점수 구체화
evaluator = load_evaluator(
    evaluator="score_string",
    criteria={"relevance": "질문과 답변의 관련성"},
    normalize_by=10,
    llm=llm
)

# DataFrame으로 변환
df_eval = pd.DataFrame(evaluated_results)

# CSV로 저장
df_eval.to_csv("KOR_IFRS_S2_2nd_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

print("✅ LLM 평가 결과 저장 완료: KOR_IFRS_S2_2nd_qa_llm_eval_results.csv")


# 코사인 유사도 검사
from langchain_openai import OpenAIEmbeddings
import key

embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


evaluated_results = []

for item in qa_results:
    try:
        answer_emb = embedding_model.embed_query(item["answer"])
        context_emb = embedding_model.embed_query(item["context"])
        score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
    except Exception as e:
        print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
        score = None
    
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "score_cosine": score
    })

import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


import pandas as pd

# 1. DataFrame으로 변환
df = pd.DataFrame(evaluated_results)

# 2. CSV로 저장
df.to_csv("KOR_IFRS_S2_2nd_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

print("✅ CSV 파일 저장 완료: KOR_IFRS_S2_2nd_qa_results_cosine_eval.csv")

169
[{'chunk_id': 'IFRS 지속가능성 공시기준 S1_목적_01', 'source': 'IFRS 지속가능성 공시기준 S1_목적', 'question': 'IFRS S1의 목적은 무엇인가요?', 'answer': 'IFRS S1의 목적은 기업의 지속가능성 관련 위험 및 기회에 대한 정보를 기업이 공시하도록 요구하여, 주요이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한 정보를 제공하는 것입니다.', 'context': '목적 1 IFRS S1 ‘지속가능성 관련 재무정보 공시를 위한 일반 요구사항’의 목적은 일반목적재무보고서의 주요이용자가 기업에 대한 자원 제공과 관련된 의사결정을 할 때 유용한, 기업의 지속가능성 관련 위험 및 기회에 대한 정보를 기업이 공시하도록 요구하는 것이 다.1) 2 지속가능성 관련 위험 및 기회에 대한 정보는 주요이용자에게 유 용하다. 이는 단기, 중기 및 장기에 걸쳐 현금흐름을 창출하는 기 업의 능력이 기업 가치사슬전반에 걸쳐 기업과 기업의 이해관계자, 사회, 경제 및 자연환경 간의 상호작용과 불가분하게 연계되어 있기 때...'}, {'chunk_id': 'IFRS 지속가능성 공시기준 S1_적용범위_01', 'source': 'IFRS 지속가능성 공시기준 S1_적용범위', 'question': '기업이 IFRS 지속가능성 공시기준을 적용할 수 있는 경우는 무엇인가요?', 'answer': '기업은 기업의 관련 일반목적재무제표가 IFRS 회계기준으로 작성되는지 또는 다른 일반적으로 인정된 회계원칙(GAAP)에 따라 작성되는지와 관계없이 IFRS 지속가능성 공시기준을 적용할 수 있습니다.', 'context': '적용범위 5 이 기준서는 IFRS 지속가능성 공시기준에 따라 지속가능성 관련 재무공시를 작성하고 보고할 때 적용한다. 6 기업 전망에 영향을 미칠 것으로 합리적으로 예상할 수 없는 지속 가능성 관련 위험 및 기회는 이 기준서의 적용범위에서 제외된다. 7 다른 IFRS 지속가능성 공시기준에서는 기업이 

,chunk_id,score,question,answer
0,IFRS 지속가능성 공시기준 S1_목적_01,1,IFRS S1의 목적은 무엇인가요?,IFRS S1의 목적은 기업의 지속가능성 관련 위험 및 기회에 대한 정보를 기업이 ...
1,IFRS 지속가능성 공시기준 S1_적용범위_01,1,기업이 IFRS 지속가능성 공시기준을 적용할 수 있는 경우는 무엇인가요?,기업은 기업의 관련 일반목적재무제표가 IFRS 회계기준으로 작성되는지 또는 다른 일...
2,IFRS 지속가능성 공시기준 S1_개념적 기반_01,1,지속가능성 관련 재무정보가 유용해지기 위해서는 어떤 조건을 충족해야 하는가요?,"지속가능성 관련 재무정보가 유용해지기 위해서는 비교가능하고, 검증가능하며, 적시성 ..."
3,IFRS 지속가능성 공시기준 S1_공정한 표시_01,1,기업이 지속가능성 관련 위험 및 기회를 공정하게 표시하기 위해 어떤 기준을 적용해야...,기업이 지속가능성 관련 위험 및 기회를 공정하게 표시하기 위해 문단 B1~B12를 ...
4,IFRS 지속가능성 공시기준 S1_중요성_01,1,기업이 중요한 정보를 식별하고 공시하기 위해 어떤 문단을 적용해야 하는가요?,기업은 중요한 정보를 식별하고 공시하기 위해 문단 B13~B37을 적용해야 합니다.


This chain was only tested with GPT-4. Performance may be significantly worse with other models.


✅ LLM 평가 결과 저장 완료: KOR_IFRS_S2_2nd_qa_llm_eval_results.csv


,chunk_id,score_cosine,question
141,IFRS 지속가능성 공시기준 S1_결론도출근거_지침의 원천_02,0.976983,기업이 지속가능성 관련 위험 및 기회를 식별하기 위해 어떤 기준과 지침을 참조할 것...
57,IFRS 지속가능성 공시기준 S1_부속지침_주요이용자_01,0.975898,IG1 IFRS S1의 목적은 무엇인가요?
147,IFRS 지속가능성 공시기준 S1_결론도출근거_지침의 원천_08,0.973293,기업이 지속가능성 관련 위험 및 기회를 식별할 때 어떤 기준을 이용해야 하는가요?
59,IFRS 지속가능성 공시기준 S1_부속지침_주요이용자의 정보수요 충족_02,0.971372,기업이 정보수요를 충족시키기 위해 어떤 유형의 이용자에게 초점을 맞춰야 하는가?
155,IFRS 지속가능성 공시기준 S1_결론도출근거_비교정보_02,0.968942,BC150 비교정보의 수정에 대한 요구사항은 무엇인가요?
77,IFRS 지속가능성 공시기준 S1_결론도출근거_도입_01,0.968196,IFRS S1은 어떤 목적을 가지고 개발되었는가?
110,IFRS 지속가능성 공시기준 S1_결론도출근거_공정한 표시_01,0.968030,IFRS S1 문단 11에서는 어떤 내용을 요구하고 있는가?
19,IFRS 지속가능성 공시기준 S1_지침의 원천_02,0.967279,기업이 지속가능성 관련 재무공시를 작성할 때 어떤 기준을 적용해야 하는지에 대해 어...
161,IFRS 지속가능성 공시기준 S1_결론도출근거_판단_02,0.967109,ISSB가 기업에게 요구하는 것은 무엇인가요?
138,IFRS 지속가능성 공시기준 S1_결론도출근거_지표 및 목표_02,0.965274,ISSB가 기업에게 요구하는 정보 공시 기준은 무엇인가요?


✅ CSV 파일 저장 완료: KOR_IFRS_S2_2nd_qa_results_cosine_eval.csv
